In [ ]:
!pip install nltk

import nltk
from nltk.chat.util import Chat, reflections


In [ ]:
!pip install requests beautifulsoup4


In [ ]:
!pip install firebase

In [ ]:
from firebase import firebase

FBconn = firebase.FirebaseApplication('https://first-db-cd01d-default-rtdb.firebaseio.com/', None)


In [ ]:
import requests
from bs4 import BeautifulSoup

def fetch_page(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup
    else:
        return None

In [ ]:
import re

def index_words(soup):
    index = {}
    words = re.findall(r'\w+', soup.get_text())
    for word in words:
        word = word.lower()
        if word in index:
            index[word] += 1
        else:
            index[word] = 1
    return index

In [ ]:
def remove_stop_words(index):

    stop_words = {'that','you','of','n','see','type','also','all','a', 'an', 'the', 'and', 'or', 'in', 'on', 'at', 'to','context'}
    for stop_word in stop_words:
        if stop_word in index:
            del index[stop_word]
    return index

In [ ]:
from nltk.stem import PorterStemmer

def apply_stemming(index):
    stemmer = PorterStemmer()
    stemmed_index = {}
    for word, count in index.items():
        stemmed_word = stemmer.stem(word)
        if stemmed_word in stemmed_index:
            stemmed_index[stemmed_word] += count
        else:
            stemmed_index[stemmed_word] = count
    return stemmed_index

In [ ]:
def search(query, index):

    stemmer = PorterStemmer()
    query_words = re.findall(r'\w+', query.lower())
    print(query)
    results = {}
    for word in query_words:
        word = stemmer.stem(word)
        if word in index:
            results[word] = index[word]
    return results

In [ ]:
def search_engine(url, query):
    soup = fetch_page(url)
    if soup is None:
        return None
    index = index_words(soup)
    index = remove_stop_words(index)
    index = apply_stemming(index)
    results = search(query, index)
    return results

In [ ]:
def sort_index_by_values(index):
    return dict(sorted(index.items(), key=lambda item: item[1], reverse=True))

In [ ]:
import string

def word_count_to_dict(text):
    # Normalize the string
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Split the string into words
    words = text.split()

    # Initialize the dictionary
    word_count = {}

    # Count the occurrences of each word
    for word in words:
        if word in word_count:
            word_count[word] += 1
        else:
            word_count[word] = 1

    return word_count

In [ ]:
# Main execution
url = 'https://cad.onshape.com/help/Content/Glossary/glossary.htm?tocpath=_____19'
soup = fetch_page(url)
if soup is None:
    print("Failed to fetch the page.")
else:
    print("Page fetched successfully.")

index = index_words(soup)
index = remove_stop_words(index)
index = apply_stemming(index)
sorted_index = sort_index_by_values(index)

quarry1="the project on schedule projct  projec?"
quarry1=word_count_to_dict(quarry1)
quarry1=remove_stop_words(quarry1)
quarry1=apply_stemming(quarry1)
print(quarry1)


index10 = {}
count = 0
for key in sorted_index:
      if count >= 10:
          break
      index10[key] = sorted_index[key]
      count += 1



Page fetched successfully.
{'project': 1, 'schedul': 1, 'projct': 1, 'projec': 1}


In [ ]:
# Assuming the resultGet contains the response
resultGet = FBconn.get('/tut6/', None)

# Extract the inner dictionary
inner_dict = list(resultGet.values())[0]
inner_dict=sort_index_by_values(inner_dict)
# Now you can work with inner_dict as a regular dictionary
print(inner_dict)

xAxis = [key for key, value in inner_dict.items()]
yAxis = [value for key, value in inner_dict.items()]
print(xAxis)
print(yAxis)




In [ ]:

import matplotlib.pyplot as plt


fig = plt.figure(figsize=(10, 6))
plt.bar(xAxis, yAxis, color='#84CFB3',width=0.8)
plt.title("Word Frequency")
plt.xlabel("Variable")
plt.xticks(rotation=360-45)
plt.ylabel("Value")
plt.show()


In [ ]:
cloud_explanation = """Private Cloud: A private cloud is a cloud computing environment that is dedicated solely to one organization.
It is usually managed either by the organization itself or by a third-party service provider. The infrastructure and services are not shared
with other organizations, offering greater control, security, and customization options. Private clouds are typically used by large enterprises
or organizations with specific compliance or security requirements.
Public Cloud: A public cloud is a cloud computing environment that is hosted and managed by a third-party service provider and made available
to multiple organizations or individuals over the internet. The infrastructure and resources are shared among multiple users,
 offering scalability, flexibility, and cost-effectiveness. Public clouds are accessible to anyone who wants to use them and are often used
 for general-purpose computing needs such as web hosting, development, and testing.
Hybrid Cloud: A hybrid cloud is a computing environment that combines elements of both private and public clouds. It allows organizations
 to leverage the benefits of both environments by integrating on-premises infrastructure with public cloud resources. This integration enables
 data and applications to be dynamically moved between the two environments based on changing workload demands, security requirements, and cost
 considerations. Hybrid clouds offer greater flexibility, scalability, and customization options compared to using either private or public clouds alone.
 They are commonly used by organizations that want to retain control over sensitive data while taking advantage of the scalability and cost-effectiveness of
 public cloud services."""

In [ ]:

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')


# Define some patterns and responses
patterns = [
    (r'the project on schedule', ['yes the project is going great', 'the project is finished', 'behind schedule']),
    (r'how are you?', ['I\'m good, thank you!', 'I\'m doing well, thanks for asking.']),
    (r'what is your name?', ['You can call me ChatGPT.', 'I go by the name ChatGPT.']),
    (r'my name is (.*)', ['glad to meet you.', 'nice to meet you.']),
    (r'thank you (.*)', ['welcome!.', 'happy to help!.']),
    (r'What is the purpose of the system ?', ['I can show you interesting statistics About your employees']),
    (r'what is cloud|cloud', ['AWS and Azure are cloud computing platforms offering scalable resources and services over the internet. They provide solutions for computing power, storage, and various other IT needs on-demand.']),
    (r'what is onshape|onshape|on shape', ['Onshape is a cloud-based CAD platform for product design and development. Learn more here: https://www.onshape.com/']),

    # You can add more patterns and responses here
]

# Create a chatbot
chatbot = Chat(patterns, reflections)

# Start the conversation
print("Hello! I'm ChatGPT. How can I help you today?")
while True:
    user_input = input("You: ")

    response = chatbot.respond(user_input)
    print("ChatGPT:", response)
    if user_input.lower() == 'exit' or user_input.lower() == 'bye':
        break






In [ ]:
pip install v-diffusion-pytorch